# 上下文管理器和else

with语句可以创建一个临时的上下文管理器对象管理上下文。with语句使得一些代码的运行更安全，例如最常见的，利用with语句读写文件，with语句能够正确的启动以及关闭数据流。

else子句除了在if语句中使用外，还能够和for、while以及try语句搭配使用。for/else、while/else和try/else中的else子句和if/else中的else子句相差甚远，因此有必要详细讨论这一区别。

## else子句

else子句和不同的语句搭配有不同的效果：

1. if
    仅当if块以及elif块的条件均不满足时才运行else子句
2. for
    仅当for循环运行完毕时（for循环没有被break终止）才运行else子句
3. while
    仅当while循环正常退出时（while循环没有被break终止）才运行else子句
4. try
    仅当try块中没有异常抛出时才运行else块，并且else块中抛出的异常不会由前面的except子句处理

从上述描述可以发现，仅有if语句中else子块表示互斥的概念，在if/else中else子块具有排他性。其他三个语句中均表示了先后的关系，即先完成前置的循环/测试然后再运行else子句中的代码

else子句在除if/else之外的语句中使用略显多余，并且其语义和惯常理解的语义有非常大的区别。当然，else子句还是有一些用途的

例如在循环中可以使用else语句代替if，以避免设置多余的控制标识，例如下述的char_check例子。

In [3]:
def char_check1(input_text):
    char_flag = False
    for _char in input_text:
        if _char == "a":
            char_flag = True
            break
    
    if not char_flag:
        print("没有发现字母a")

def char_check2(input_text):
    for _char in input_text:
        if _char == "a":
            break
    else:
        print("没有发现字母a")

test_text = "dsfcvsd"
char_check1(test_text)
char_check2(test_text) 


没有发现字母a
没有发现字母a


在try语句中else子句承担逻辑上的要求：try语句用于捕获特定代码可能抛出的错误，因此try块不应该捕获后续代码抛出的错误，此时应当在else语句中运行后续代码。本书有一个非常好的例子用于说明try/else语句。下述伪代码中dangerous_call()表示应当捕获错误的代码，after_call()表示后续代码：

不正确的使用方式如下：
```Python
try:
    dangerous_call()
    after_call()
except ErrorType:
    ...
```
正确的使用方式：
```Python
try:
    dangerous_call()
except ErrorType:
    ...
else:
    after_call()
```

## EAFP & LBYL

EAFP和LBYL是两种风格，表示了处理错误以及控制流程的方式。

* EAFP(easier to ask for forgiveness than permission)
不管如何，先运行，然后处理错误。
Python中常用这种方式。很多Python程序会首先假定代码能够正常运行，若假定不成立，抛出错误，那就捕获这个错误，然后处理。
* LBYL(look before you leap)
不同于EAFP，LBYL在运行特定代码前会执行各类检查以及分派以尽可能确保代码能够正常运行。

LBYL在多线程中可能引起意外错误。例如通过if检查某个键是否存在，然后从字典中读取该键对应的值；对于多线程，很可能在线程A中通过了if检查，但是线程B删除了该键值对导致运行失败

## with与上下文管理器

with语句旨在简化try/finally模式。with语句能够保证一段代码在运行前以及运行后执行某些操作，无论这段代码是否正常运行。显然，这种逻辑能够释放一些重要的资源，或者还原临时变更的状态。

with语句会创建一个上下文管理器用于管理和实施上述操作。具体来说，上下文管理器会在with语句开始时调用__enter__方法，并在with语句结束时调用__exit__方法。值得注意的是，不同于函数，with语句并没有定义新的作用域，with语句中创建的变量在with语句结束后依然可以调用。

本章给出了一个非常好的例子，该例子用于说明with语句的应用以及一些注意事项：

In [4]:
class LookingGlass:

    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return "JABBERWOCKY"
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
    
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print("Please DO NOT divide by zero!")
            return True

with LookingGlass() as what:
    print("Alice, Kitty and snowdrop")
    print(what)
print("Alice, Kitty and snowdrop")
print(what)

pordwons dna yttiK ,ecilA
YKCOWREBBAJ
Alice, Kitty and snowdrop
JABBERWOCKY


上述例子揭示了with语句的很多注意事项：
1. with语句开始时会创建上下文管理器并且调用上下文管理器的__enter__方法
2. __enter__方法完全可以修改环境，而不仅仅是进行一些数据/数据流上的工作，例如上述例子中利用自定义的函数接管了标准输出方式
3. __enter__方法的返回值可以任选，并不一定要返回上下文管理器自身，甚至可以不返回任何东西，虽然在很多应用中都是返回self
4. __exit__方法可以接收with语句运行过程中抛出的错误并对这些错误进行处理

## contextlib中的实用工具

contextlib模块提供了一些适用于特定场合的类和函数，本章列举了一些。

1. closing
若对象提供了close()方法，而没有实现__enter__/__exit__，则可以使用closing函数构建一个上下文管理器。显然closing()函数适用于仅在退出with语句时进行特殊处理的应用场合
2. suppress
suppress可用于构建临时忽略指定异常的上下文管理器
3. @contextmanager
该装饰器可以将生成器函数变为上下文管理器，从而避免创建类
4. ContextDecorator
ContextDecorator是个基类，用于定义基于类的上下文管理器
5. ExitStack
能够管理多个上下文管理器的__exit__运行顺序 —— 按照先进后出的顺序调用各上下文管理器的__exit__方法。具体来说ExitStack可以创建管理上下文管理器的对象，该对象具有enter_context方法用于启动一个上下文管理器，在退出with语句时会按照先进后出的顺序调用这个上下文管理器的__exit__方法

### @contextmanager

@contextmanager能够将一个生成器“变为”一个上下文管理器。具体来说，被@contextmanager装饰的生成器会以yield为界，yield语句前的部分相当于__enter__方法，会在with语句开始时被调用；yield语句后的部分相当于__exit__方法，会在with语句结束时调用（吐槽：更像断点调试了）

利用@contextmanager可以将上述的LookingGlass改写为如下函数，功能完全一致。

In [7]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write

    msg = ""
    try:
        yield "JABBERWOCKY"
    except ZeroDivisionError:
        msg = "Please DO NOT divide by zero!"
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

with looking_glass() as what:
    print("Alice, Kitty and snowdrop")
    print(what)
print("Alice, Kitty and snowdrop")
print(what)

pordwons dna yttiK ,ecilA
YKCOWREBBAJ
Alice, Kitty and snowdrop
JABBERWOCKY


上述with语句的调用过程如下：

1. @contextmanager会将生成器包装成实现__enter__和__exit__方法
2. 对于__enter__方法
首先会调用生成器函数并保存生成器对象gen
然后调用next(gen)并且获取返回值以绑定到with/as中
3. 对于__exit__方法
首先会检查是否有抛出异常，若抛出异常则调用gen.throw(exception)并在yield位置抛出
若没有抛出异常则继续运行next(gen)以运行yield语句后的部分

值得注意的是，若在生成器中没有使用try/finally语句处理在yield位置抛出的错误，那么yield后续的代码不会运行。对于looking_glass，yield后续代码无法运行意味着无法恢复标准输出方式，这可能导致致命问题。

## 总结

1. else子句和不同的语句搭配会有不同的效果，在if语句中else子句表示了一种互斥的概念，而在for/while/try语句中else子句则表示了一种先后关系
2. try语句中的else子句有特殊的作用。为了尽可能仅对特定代码抛出的错误进行捕获和处理，try语句中应当仅放入特定代码，其余代码则放置到else语句
3. EAFP和LBYL是两种代码风格，Python作为一种弱类型语言，其趋向于使用EAFP，即首先假定代码能正常运行，若运行过程中报错则处理错误
4. with语句的正确使用能够创建上下文管理器接管一些重要设置的开启和关闭。with语句能够保证在一段代码执行前后执行一些必要操作，这些操作能够获取/释放一些重要资源，或者改变/还原临时变更的状态
5. with语句创建的上下文管理器依靠__enter__和__exit__执行相应的功能。在进入with语句时，解释器会自动调用并运行上下文管理器的__enter__方法，在退出with语句时，解释器则会自动调用并运行上下文管理器的__exit__方法
6. __enter__方法可以有返回值，也可以没有返回值；__exit__方法则可以接收并处理with语句运行过程中抛出的错误
7. contextlib模块中提供了一些适用于特定场合的类以及函数。其中@contextmanager管理器能够将一个生成器变为一个上下文管理器。具体来说，以yield为界，yield语句前的部分相当于__enter__方法，会在with语句开始时被调用；yield语句后的部分相当于__exit__方法，会在with语句结束时调用。值得注意的是，若yield之前的语句抛出错误，yield之后的语句不会运行，这在一些应用中可能导致致命错误